In [26]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))



#fig = px.pie(df, values='pop', names='country', title='Population of European continent')


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')


# get the base64 value of the image
image_filename = "logo.png"
import base64
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A(href="https://www.snhu.edu/", children=[
        html.Img(src="data:image/png;base64,{}".format(encoded_image.decode()), alt="Grazioso Logo", width=200, height=200),
    ]),
    
    
    html.Center([
        html.B(html.H1('SNHU CS-340 Dashboard')),
        html.H3('Hello from Kyle Keiper')
    ]),
    html.Hr(),
    
    # I would like to use dcc.RadioItems, but the method is broken. I get some 
    # "TypeError: unsupported format string passed to list.__format__"
    # dcc.RadioItems(['Water Rescue', 'Mountain Rescue', 'Disaster Rescue', 'Reset'], 'Reset')
    # this is the EXACT format on the radio items page (https://dash.plotly.com/dash-core-components/radioitems)
    # so not saying Dash and Plotly suck, but Dash and Plotly suck
    
    html.Div(
        className='row',
        style={ 'display': 'flex' },
        children=[            
            # I know this isn't right, since I can't seem to just use the name attribute in the callback, this is broken
            # in PHP, this would be a simple as just doing `$value = $_POST['rescue-type']`
            # but python doesn't make anything easy. It sucks.
            
            dcc.Input(type="radio", name='rescue-type', id="submit-button-one", value="water"),
            "Water Rescue", 
            
            dcc.Input(type="radio", name='rescue-type', id="submit-button-two", value="mountain"),
            "Mountain Rescue",
            
            dcc.Input(type="radio", name='rescue-type', id="submit-button-three", value="disaster"),
            "Disaster Rescue",
            
            dcc.Input(type="radio", name='rescue-type', id="submit-button-four", value="reset"),
            "Reset"
        ]
    ),

    # again, pulling stuff STRAIGHT from the documentation is broken. Dash sucks.
    # https://dash.plotly.com/basic-callbacks#dash-app-with-multiple-inputs
    
    #dcc.RadioItems(
    #    ['Linear', 'Log'],
    #    'Linear',
    #    id='xaxis-type',
    #    inline=True
    #),

    html.Div(
        id="display",
        className="row",
        style={"display": "flex"},
        children=[]
    ),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,
        sort_action="native",
        page_action='native',
        filter_action="native",
        page_current= 0,
        page_size= 50,
    ),

    
    html.Br(),
     html.Hr(),
     html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it



@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    
    
    
    return [
        dcc.Graph(            
            figure = {
                'data': pd.DataFrame.from_dict(viewData).to_dict('records'),
                'layout': {
                    'title': "My DCC Chart"
                }
            }
        )    
    ]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip("Tooltip"),
                # dff.iloc[0,4]
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P("Test name")
                    # dff.iloc[0,1]
                ])
            ])
        ])
    ]
    
# this was my attempt to find out how to grab the argument value based on the input-name, not input id
# never worked, but would would work fine in PHP
@app.callback(
    Output("display", "children"),
    [
        Input("submit-button-one", "value"),
        Input("submit-button-two", "value"),
        Input("submit-button-three", "value"),
        Input("submit-button-four", "value")
    ]
)
def show_values(v1, v2, v3, v4):
    return "Values: {}, {}, {}, {}".format(v1, v2, v3, v4)
    
    
@app.callback(
    Output("datatable-id", "data"),
    [
        Input("submit-button-one", "value"),
        Input("submit-button-two", "value"),
        Input("submit-button-three", "value"),
        Input("submit-button-four", "value")
    ]
)
def cb_render(water, mountain, disaster, reset):
    
    dataset = pd.DataFrame.from_records(shelter.read({}))
    
    if(water):
        dataset = pd.DataFrame.from_records(shelter.read({
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }))
        
    elif(mountain):
        dataset = pd.DataFrame.from_records(shelter.read({
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }))
        
    elif(disaster):
        dataset = pd.DataFrame.from_records(shelter.read({
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }))
    
    return dataset.to_dict('records')



# ideally, I would love to do something like the following, but dash doesn't seem to expose a way to hook this up
def correct_callback(rescue_type):
    # default is all types
    dataset = pd.DataFrame.from_records(shelter.read({}))
    
    # water rescue parameters
    if(rescue_type is "water"):
        dataset = pd.DataFrame.from_records(shelter.read({
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }))
        
    # mountain rescue parameters
    elif(rescue_type is "mountain"):
        dataset = pd.DataFrame.from_records(shelter.read({
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }))
        
    # disaster recovery parameters
    elif(rescue_type is "disaster"):
        dataset = pd.DataFrame.from_records(shelter.read({
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }))
    
    # finally, take whatever value we have in dataset and turn it into a dict for dash
    return dataset.to_dict('records')

app